In [4]:
import requests
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()

bearer_token = os.getenv('BEARER_TOKEN')

# endpoint --> aletheia
url = 'https://api.aletheiaapi.com/StockData'

# authorization header
headers = {
    'Authorization': f'Bearer {bearer_token}'
}
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print(data)  # You can process the data further as needed
else:
    print('Failed to retrieve data:', response.status_code)

Failed to retrieve data: 401


In [7]:
response

<Response [401]>